In [24]:
import geopandas as gpd
import pandas as pd
import folium
import matplotlib.pyplot as plt
import seaborn as sns
sf = gpd.read_file("../data/raw/VMADMIN/POSTCODE_POLYGON.shp")
sf['centroid'] = sf['geometry'].apply(lambda x: (x.centroid.y, x.centroid.x))
#read data
df = pd.read_csv('../data/curated/price_normalised3.csv')
df.head()

,Unnamed: 0,name,postcode,property_type,bedrooms,bathrooms,parking,schools,cost_text,train_station(m),CBD(m),population2022,population2023,population2024,population2025,income2022,income2023,income2024,income2025,price_label
0,5321,Room 6/8 Melwood Court Meadow Heights VIC 3048,3048,Apartment / Unit / Flat,1,1,0,0.3,130.0,2131.1,19150.7,17364.0,16549.0,15773.0,15034.0,86498,88316,90176,92078,0
1,5569,30 Daly Boulevard Highton VIC 3216,3216,House,9,2,2,2.0,130.0,2904.1,85109.1,59229.0,60753.0,62377.0,64109.0,184195,192855,201927,211432,0
2,640,62 Reserve Rd Grovedale VIC 3216,3216,House,1,2,2,1.0,140.0,1809.3,87665.1,59229.0,60753.0,62377.0,64109.0,184195,192855,201927,211432,0
3,5570,1/57 Wanda Road Caulfield North VIC 3161,3161,House,1,1,0,0.6,140.0,1432.7,9784.5,10418.0,10185.0,9957.0,9734.0,56454,58176,59951,61780,0
4,5571,Room 2/31 Stanley Street Frankston VIC 3199,3199,House,1,1,1,1.1,145.0,916.1,53197.1,60500.0,59711.0,58932.0,58164.0,176387,182176,188161,194348,0


In [25]:
#group the data by postcode
df_1 = df.groupby('postcode').mean()

In [26]:
postcode = []
for i in sf.iloc[:,1:2].values:
    postcode.append(int(i))
sf['postcode'] = postcode
# create a JSON 
geoJSON = sf[['postcode', 'geometry']].drop_duplicates('postcode').to_json()

In [27]:
df_affordable = df_1.sort_values(by='cost_text', axis=0, ascending=True)
df_affordable['postcode'] = df_affordable.index
df_top5_affordable = df_affordable.head()
df_top5_affordable[['cost_text']]
# top  5 suburb has lowest rent
# affordabel for poor family

,cost_text
postcode,
3717,250.000000
3714,280.000000
3085,289.908524
3061,322.500000
3175,332.230769


In [28]:
post_code = []
for i in sf['POSTCODE']:
    post_code.append(int(str(i)))
sf['postcode'] = post_code
df_top5_affordable = df_top5_affordable.reset_index(drop=True)
#merge the two dataset with the same postcode
df_top5_affordable= pd.merge(df_top5_affordable,sf,on = ['postcode'],how='left')

In [29]:
m2 = folium.Map(location=[-37.0,144], control_scale=True, zoom_start=7)

# The average rent of each suburb
c2 = folium.Choropleth(
    geo_data=geoJSON,
    name='choropleth2', 
    data=df_affordable, 
    columns=['postcode','cost_text'],
    key_on='properties.postcode',
    fill_color='YlOrRd',
    nan_fill_color='black',
    legend_name='num_of_properties'
)

c2.add_to(m2)

In [30]:
df_copy = df_top5_affordable.copy()

for i in range(0, len(df_copy)):
    html="""
    <h4> Postcode: </h4>""" + str(df_copy.iloc[i]['postcode'])+ \
    """<h4>Growth_rate:</h4>""" + str(df_copy.iloc[i]['cost_text'])
    iframe = folium.IFrame(html=html, width=200, height=300)
    popup = folium.Popup(iframe, max_width=2650)
    folium.Marker(
        location=[df_copy.iloc[i]['centroid'][0],df_copy.iloc[i]['centroid'][1]],
        popup= popup,
        tooltip=str(df_copy.iloc[i]['postcode']),
        icon=folium.Icon(color='green',icon='home',prefix="fa"),
    ).add_to(m2)

In [31]:
#save plot
import os
fig_save_path = "../plots/"
if not os.path.exists(fig_save_path):
    os.makedirs(fig_save_path)
m2.save(os.path.join(fig_save_path,"top 5 postcodes with lowest rent.html"))